# 🧠 PAMPAr-Coder 1.75B - Monitor de Entrenamiento

**Modelo**: PAMPAr-Coder 1.75B params (52 Brodmann Zones, 4 Territorios, 24 bloques)  
**Hardware**: RunPod A40 46GB VRAM  
**Config**: dim=1024, heads=16, capas=24, seq=512, batch=2, grad_accum=32  
**Log**: `/workspace/PAMPAr-Coder/training_1_75B.log`

Ejecutá las celdas en orden. La última celda tiene auto-refresh cada 30s.

## 1. Setup - Importar librerías y configurar conexión SSH

In [ ]:
import subprocess
import re
import time
from datetime import datetime, timedelta
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

# Estilo oscuro para los gráficos
plt.style.use('dark_background')
plt.rcParams.update({
    'figure.figsize': (14, 5),
    'axes.grid': True,
    'grid.alpha': 0.3,
    'font.size': 11,
    'axes.facecolor': '#1a1a2e',
    'figure.facecolor': '#16213e',
})

# ============================================================
# CONFIGURACIÓN SSH AL POD
# ============================================================
POD_HOST = "69.30.85.114"
POD_PORT = "22072"
POD_USER = "root"
POD_PASS = "Snuggles2020"
LOG_FILE = "/workspace/PAMPAr-Coder/training_1_75B.log"

# Modelo info
MODEL_PARAMS = 1.75e9
STEPS_PER_EPOCH = 113457
TOTAL_EPOCHS = 30

def ssh_cmd(command: str, timeout: int = 15) -> str:
    """Ejecuta un comando en el pod via SSH."""
    try:
        result = subprocess.run(
            ["sshpass", "-p", POD_PASS, "ssh",
             "-o", "StrictHostKeyChecking=no",
             "-o", "ConnectTimeout=10",
             "-p", POD_PORT, f"{POD_USER}@{POD_HOST}",
             command],
            capture_output=True, text=True, timeout=timeout
        )
        return result.stdout.strip()
    except subprocess.TimeoutExpired:
        return "TIMEOUT"
    except FileNotFoundError:
        # Fallback sin sshpass (usa ssh directo, requiere key auth o agent)
        try:
            result = subprocess.run(
                ["ssh", "-o", "StrictHostKeyChecking=no",
                 "-o", "ConnectTimeout=10",
                 "-p", POD_PORT, f"{POD_USER}@{POD_HOST}",
                 command],
                capture_output=True, text=True, timeout=timeout
            )
            return result.stdout.strip()
        except Exception as e:
            return f"ERROR: {e}"

# Test de conexión
test = ssh_cmd("echo 'Conectado al pod!'")
print(f"🔗 SSH: {test}")
print(f"📍 Pod: {POD_USER}@{POD_HOST}:{POD_PORT}")

## 2. Obtener datos del log y GPU

In [ ]:
def fetch_training_data():
    """Obtiene y parsea el log de entrenamiento del pod."""
    raw = ssh_cmd(f"cat {LOG_FILE}")
    
    if "ERROR" in raw or "TIMEOUT" in raw:
        print(f"⚠️ Error conectando: {raw}")
        return [], [], []
    
    steps = []
    losses = []
    grads = []
    
    # Parsear líneas de tqdm: "Training:  X%|...| STEP/TOTAL [..., loss=X.XXXX, grad=X.XX]"
    pattern = r'\|\s*(\d+)/\d+\s*\[.*?loss=([\d.]+)(?:.*?grad=([\d.]+|N/A))?'
    
    for match in re.finditer(pattern, raw):
        step = int(match.group(1))
        loss = float(match.group(2))
        grad_str = match.group(3)
        grad = float(grad_str) if grad_str and grad_str != 'N/A' else None
        
        # Evitar duplicados (tqdm actualiza la misma línea)
        if not steps or step != steps[-1]:
            steps.append(step)
            losses.append(loss)
            grads.append(grad)
    
    return steps, losses, grads


def fetch_gpu_stats():
    """Obtiene estadísticas de la GPU."""
    raw = ssh_cmd("nvidia-smi --query-gpu=memory.used,memory.total,temperature.gpu,utilization.gpu,power.draw --format=csv,noheader,nounits")
    if "ERROR" in raw or "TIMEOUT" in raw:
        return None
    
    try:
        parts = [x.strip() for x in raw.split(',')]
        return {
            'mem_used': int(parts[0]),
            'mem_total': int(parts[1]),
            'temp': int(parts[2]),
            'util': int(parts[3]),
            'power': float(parts[4]),
        }
    except:
        return None


def fetch_process_info():
    """Verifica si el proceso de training sigue corriendo."""
    raw = ssh_cmd("ps aux | grep 'train_cloud.py' | grep -v grep | head -1")
    if raw and "train_cloud" in raw:
        # Extraer tiempo de ejecución
        etime = ssh_cmd("ps -p $(pgrep -f train_cloud.py | head -1) -o etime= 2>/dev/null")
        return True, etime.strip()
    return False, "N/A"


# Test: obtener datos
steps, losses, grads = fetch_training_data()
gpu = fetch_gpu_stats()
running, elapsed = fetch_process_info()

print(f"📊 Datos obtenidos: {len(steps)} puntos de datos")
print(f"🏃 Proceso activo: {'SI' if running else 'NO'} | Tiempo: {elapsed}")
if gpu:
    print(f"🖥️ GPU: {gpu['mem_used']}/{gpu['mem_total']} MiB | {gpu['temp']}°C | {gpu['util']}% | {gpu['power']}W")
if losses:
    print(f"📉 Loss: {losses[0]:.4f} → {losses[-1]:.4f} (último step: {steps[-1]})")

## 3. Visualizar curvas de Loss y Gradient Norm

In [ ]:
def plot_training(steps, losses, grads, gpu_stats=None, running=True, elapsed=""):
    """Genera los gráficos de monitoreo."""
    
    if not steps:
        print("⏳ Sin datos de entrenamiento aún...")
        return
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle(f'PAMPAr-Coder 1.75B — Entrenamiento en A40', 
                 fontsize=14, fontweight='bold', color='#e94560')
    
    # =============================================
    # 1. LOSS
    # =============================================
    ax1 = axes[0]
    ax1.plot(steps, losses, color='#0f3460', linewidth=1.5, alpha=0.4, label='Raw')
    
    # Smoothed loss (media móvil)
    if len(losses) > 20:
        window = min(50, len(losses) // 5)
        smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
        smooth_steps = steps[window-1:]
        ax1.plot(smooth_steps, smoothed, color='#e94560', linewidth=2.5, label=f'Smooth (w={window})')
    
    ax1.set_xlabel('Steps')
    ax1.set_ylabel('Loss')
    ax1.set_title(f'Loss: {losses[-1]:.4f}', fontsize=12)
    ax1.legend(loc='upper right', fontsize=9)
    
    # Marcar epochs
    for e in range(1, 31):
        epoch_step = e * STEPS_PER_EPOCH
        if epoch_step <= steps[-1]:
            ax1.axvline(x=epoch_step, color='yellow', alpha=0.2, linestyle='--')
    
    # =============================================
    # 2. GRADIENT NORM
    # =============================================
    ax2 = axes[1]
    valid_grads = [(s, g) for s, g in zip(steps, grads) if g is not None]
    if valid_grads:
        g_steps, g_vals = zip(*valid_grads)
        ax2.plot(g_steps, g_vals, color='#533483', linewidth=1, alpha=0.5, label='Raw')
        
        if len(g_vals) > 20:
            window = min(50, len(g_vals) // 5)
            smoothed_g = np.convolve(g_vals, np.ones(window)/window, mode='valid')
            ax2.plot(g_steps[window-1:], smoothed_g, color='#e94560', linewidth=2, label='Smooth')
        
        ax2.set_ylabel('Gradient Norm')
        ax2.set_title(f'Grad Norm: {g_vals[-1]:.2f}', fontsize=12)
        ax2.legend(loc='upper right', fontsize=9)
    ax2.set_xlabel('Steps')
    
    # =============================================
    # 3. ESTADO / GPU
    # =============================================
    ax3 = axes[2]
    ax3.axis('off')
    
    current_epoch = steps[-1] / STEPS_PER_EPOCH
    progress_pct = (steps[-1] / (STEPS_PER_EPOCH * TOTAL_EPOCHS)) * 100
    
    # Calcular velocidad y ETA
    if len(steps) > 10:
        recent_steps = steps[-10:]
        # Estimar it/s del log de tqdm
        speed_raw = ssh_cmd(f"tail -1 {LOG_FILE} | grep -oP '[\\d.]+it/s'")
        speed = speed_raw if speed_raw else "?"
    else:
        speed = "?"
    
    status_color = '#00d2ff' if running else '#ff4444'
    status_text = '🟢 ENTRENANDO' if running else '🔴 DETENIDO'
    
    info_lines = [
        f"Estado: {status_text}",
        f"Tiempo: {elapsed}",
        f"",
        f"Step: {steps[-1]:,} / {STEPS_PER_EPOCH * TOTAL_EPOCHS:,}",
        f"Epoch: {current_epoch:.2f} / {TOTAL_EPOCHS}",
        f"Progreso: {progress_pct:.2f}%",
        f"Velocidad: {speed}",
        f"",
        f"Loss actual: {losses[-1]:.4f}",
        f"Loss inicial: {losses[0]:.4f}",
        f"Reducción: {((losses[0]-losses[-1])/losses[0]*100):.1f}%",
    ]
    
    if gpu_stats:
        info_lines.extend([
            f"",
            f"GPU VRAM: {gpu_stats['mem_used']}/{gpu_stats['mem_total']} MiB",
            f"GPU Temp: {gpu_stats['temp']}°C",
            f"GPU Util: {gpu_stats['util']}%",
            f"GPU Power: {gpu_stats['power']}W",
        ])
    
    info_text = '\n'.join(info_lines)
    ax3.text(0.05, 0.95, info_text, transform=ax3.transAxes,
             fontsize=11, verticalalignment='top', fontfamily='monospace',
             color='white',
             bbox=dict(boxstyle='round,pad=0.5', facecolor='#1a1a2e', edgecolor='#e94560', alpha=0.9))
    ax3.set_title('Estado', fontsize=12)
    
    plt.tight_layout()
    plt.show()

# Ejecutar una vez
steps, losses, grads = fetch_training_data()
gpu = fetch_gpu_stats()
running, elapsed = fetch_process_info()
plot_training(steps, losses, grads, gpu, running, elapsed)

## 4. Análisis de convergencia y detección de overfitting